Домашнее задание. Условие: дана таблица с колонками (id, name, salary, manager_id). Студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров

In [1]:
!pip install pyspark >> None

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import numpy as np

Способ решения #1 (с помощью sql-запроса)

In [5]:
# Создаем Spark сессию
spark = SparkSession.builder.appName("CompareSalary1").getOrCreate()

In [6]:
# Получаем исходные данные
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],'Salary': ['70', '80', '60', '90'], 'Manager_id': ['3', '4', 'Null', 'Null']}

In [7]:
# Создаем DataFrame из данных
df = spark.createDataFrame([(data['id'][i], data['Name'][i], int(data['Salary'][i]),
                             data['Manager_id'][i]) for i in range(len(data['id']))],
                              ['id', 'Name', 'Salary', 'Manager_id'])

In [8]:
# Преобразуем столбец Salary в числовой формат
df = df.withColumn('Salary', col('Salary').cast('int'))

In [9]:
# Создадим временную таблицу.
# Срок службы этой временной таблицы привязан к текущему сеансу Spark.
df.createOrReplaceTempView("employees")

In [10]:
# Выполним вложенный в spark запрос sql для вывода строк, где зарплата сотрудника больше зарплаты менеджера
result = spark.sql("SELECT e.Name FROM employees e JOIN employees m ON e.Manager_id = m.id WHERE e.Salary > m.Salary")

In [11]:
# Выводим результат
result.show()

+----+
|Name|
+----+
| Joe|
+----+



In [12]:
# Останавливаем Spark сессию
spark.stop()

Способ решения #2 (с помощью join)

In [13]:
# Создаем Spark сессию
spark = SparkSession.builder.appName("CompareSalary2").getOrCreate()

In [14]:
# Получаем исходные данные
data = {'id': [1, 2, 3, 4],
        'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'],
        'Manager_id': ['3', '4', 'Null', 'Null']}

In [15]:
# В принципе, можно обойтись и без pandas
import pandas as pd
df = pd.DataFrame(data)

In [16]:
# Создаем DataFrame из данных
df = spark.createDataFrame(list(zip(data['id'], data['Name'], data['Salary'], data['Manager_id'])), ['id', 'Name', 'Salary', 'Manager_id'])

In [17]:
# Преобразуем столбцы Salary и Manager_id в числовой формат
df = df.withColumn('Salary', col('Salary').cast('int'))
df = df.withColumn('Manager_id', col('Manager_id').cast('int'))

In [19]:
# Присоединяем DataFrame сам к себе по столбцу Manager_id и фильтруем строки, где зарплата сотрудника больше зарплаты менеджера
result = df.alias('employee').join(df.alias('manager'), col('employee.Manager_id') == col('manager.id'), 'left')\
    .filter(col('employee.Salary') > col('manager.Salary'))\
    .select(col('employee.Name').alias('E_Name'))

In [21]:
# Выводим результат
result.show()

+------+
|E_Name|
+------+
|   Joe|
+------+



In [22]:
# Останавливаем Spark сессию
spark.stop()